In [14]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
#from keras.optimizers import Adam

In [15]:
img_width, img_height = 224, 224

In [16]:
# importa o modelo ResNet50 e descarta a última camada do classifier.
model=DenseNet201(weights='imagenet',include_top=False, input_shape=(img_width, img_height, 3))

In [17]:
model.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                       

In [18]:
for layer in model.layers:
    layer.trainable = False
print(model.output)   

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 1920), dtype=tf.float32, name=None), name='relu/Relu:0', description="created by layer 'relu'")


In [19]:
last_layer = model.get_layer('conv5_block32_concat')
print(last_layer.output_shape)

(None, 7, 7, 1920)


In [20]:
x = Flatten()(last_layer.output)
x = Dense(1024, activation = 'relu')(x)
# x = Dropout(0.3)(x)
# x = Dense(512, activation = 'relu')(x)
x= Dense(10,'softmax')(x)
model = Model(model.input, x)

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [22]:
!gdown --id 1_2Bp5D8TAXe2-tPf-RcQSeouB26in3Qx

Downloading...
From: https://drive.google.com/uc?id=1_2Bp5D8TAXe2-tPf-RcQSeouB26in3Qx
To: /content/version3.zip
100% 626M/626M [00:04<00:00, 132MB/s]


In [23]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
!unzip /content/version3.zip

In [25]:
directory = '/content/content/data'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.4) # set validation split

train_generator = train_datagen.flow_from_directory(
    directory,
    #color_mode='grayscale',
    target_size=(224, 224,),
    batch_size=128,
    subset='training') # set as training data

print(directory)
validation_generator = train_datagen.flow_from_directory(
    directory, # same directory as training data
    target_size=(224, 224),
    #color_mode='grayscale',
    batch_size=128,
    subset='validation') # set as validation data    


In [27]:
import tensorflow as tf
callback = tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', 
                                              save_weights_only=False,
                                              monitor='val_accuracy',
                                              mode='max',
                                              save_best_only=True)




In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'],)

In [ ]:
history = model.fit_generator(train_generator,
                    epochs = 50,
                    validation_data = validation_generator,
                    validation_steps = 50,
                    callbacks= [callback])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/50
731/731 [==============================] - 275s 352ms/step - loss: 1.7525 - accuracy: 0.6653 - val_loss: 0.4845 - val_accuracy: 0.8330
Epoch 2/50
731/731 [==============================] - 247s 338ms/step - loss: 0.4038 - accuracy: 0.8572 - val_loss: 0.3458 - val_accuracy: 0.8867
Epoch 3/50
731/731 [==============================] - 247s 338ms/step - loss: 0.3002 - accuracy: 0.8945 - val_loss: 0.3163 - val_accuracy: 0.8947
Epoch 4/50
731/731 [==============================] - 247s 337ms/step - loss: 0.2357 - accuracy: 0.9169 - val_loss: 0.3700 - val_accuracy: 0.8822
Epoch 5/50
731/731 [==============================] - 252s 345ms/step - loss: 0.2159 - accuracy: 0.9235 - val_loss: 0.3093 - val_accuracy: 0.9023
Epoch 6/50
731/731 [==============================] - 247s 338ms/step - loss: 0.1823 - accuracy: 0.9364 - val_loss: 0.3042 - val_accuracy: 0.9041
Epoch 7/50
731/731 [==============================] - 247s 337ms/step - loss: 0.1745 - accuracy: 0.9386 - val_loss: 0.3092 -

In [ ]:
print(train_generator.classes)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Gráfico do histórico do treino
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# salvar modelo
model.save("model.h5")
print("Modelo salvo em disco dessa instância do Google Colab")

In [ ]:
# métricas de perda e acurácia
score = model.evaluate_generator(validation_set, 105)

print ("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print ("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# carregar modelo salvo no HD
from keras.models import load_model

model = load_model('model.h5')

In [ ]:
model.summary()

In [ ]:
# visualização das camadas
for i, layer in enumerate(model.layers):
   print(i, layer.name)

In [ ]:
for layer in model.layers[:167]:
   layer.trainable = False
for layer in model.layers[167:]:
   layer.trainable = True

In [ ]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.000001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from google.colab import drive
from keras.callbacks import *

drive.mount('/content/drive')
filepath="/content/drive/My Drive/Colab Notebooks/log/resnet50:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# Carregando o modelo com o peso melhor treinado (exemplo: carregando epoch 47, validation accuracy de 90.5%)

model.load_weights('/content/drive/My Drive/Colab Notebooks/log/resnet50:030-val_acc:0.629.hdf5')

In [ ]:
# Fine-tune
history = model.fit_generator(training_set,
                    steps_per_epoch=528/32,
                    epochs = 100,
                    validation_data = validation_set,
                    validation_steps = 105/32,
                    callbacks=callbacks_list)

In [ ]:
model.save("model.h5")
uploaded = drive.CreateFile({'title': 'model.h5'})
uploaded.SetContentFile('model.h5')
uploaded.Upload()

In [ ]:
import matplotlib.pyplot as plt

# listar todos os dados no history
print(history.history.keys())
# Gráfico de treino - acurácia
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# Gráfico de treino - perda
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
!pip install --upgrade --quiet PyDrive
# para conectar com o Google Drive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Importando imagens de teste do drive

link = 'https://drive.google.com/open?id=1sbjcK__NABa7gfsPOt7JM8jAaXCLCfKW'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste1_cytHigh-grade Squamous Intraepithelial Lesion - 14659.jpg')

link = 'https://drive.google.com/open?id=1vDBjgozlaLg0tcGQ50b9wpOBz_NYyx8o'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste2_cyt14721.jpg')

link = 'https://drive.google.com/open?id=14mCco19UM0k83Irdz3xk2lQaOs7a1YuK'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('teste3_cytoCandida - 7557.jpg')

In [ ]:
# Testando o modelo

test_image = image.load_img('teste1_cytHigh-grade Squamous Intraepithelial Lesion - 14659.jpg', target_size = (200, 200))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

if result[0][0] == 0:
    diagnostico = 'Carcinoma'
elif result[0][0] == 1:
    diagnostico = 'Normal'
else:
    diagnostico = 'Outros problemas'
    
print ('Diagnóstico:', diagnostico)

In [ ]:
# Salvando o modelo no drive

uploaded = drive.CreateFile({'title': 'model.h5'})
uploaded.SetContentFile('model.h5')
uploaded.Upload()

In [ ]:
# Importando o modelo do drive

link = 'https://drive.google.com/open?id=1b8QkLS1nix4K5g46SGFAR4Cnx1D6eoF1'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('model.h5')